In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("qbbo.ipynb")

In [2]:
import pandas as pd
import numpy as np
%matplotlib inline
import otter
grader = otter.Notebook()

QBBO_PATH = '04222019.QBBO_21_C-decal'

# Parsing QBBO

QBBO is NASDAQ's level 1 market data protocol. You can [read its specification online](https://www.nasdaqtrader.com/content/technicalsupport/specifications/dataproducts/QBBOSpecification2.1.pdf). This protocol consists of a repeated series of a message length then a message.

Let's start with some helper functions:

In [3]:
import datetime

def read_int(raw_bytes):
    return int.from_bytes(raw_bytes, byteorder='big')

def read_string(raw_bytes):
    return raw_bytes.decode('ascii')

def parse_timestamp(ns_since_midnight):
    midnight = datetime.datetime(2019, 4, 22)
    delta = datetime.timedelta(seconds=ns_since_midnight / 1e9)
    return midnight + delta

**Question 1:** QBBO consists of a series of messages, each consisting of two bytes for length. For example, see the following:

```
00 06 aa bb cc dd ee ff 00 04 14 01 48 03
[len] [    message    ] [len] [ message ]
```

The above contains two messages:
1. `aa bb cc dd ee ff`
2. `14 01 48 03`

A file `04222019.QBBO_21_C-decal` has been provided alongside this assignemnt. It contains real NASDAQ QBBO market data, filtered to only contain system messages and a few tickers (to reduce the size of the download).

Read this file, yielding each message's content.

In [4]:
def read_qbbo_messages(path):
    with open(path, 'rb') as f:
        while True:
            length_raw = f.read(2)
            if not length_raw:
                break # End of file
            length = read_int(length_raw)

            yield f.read(length)

In [5]:
test_reader = read_qbbo_messages(QBBO_PATH)
assert next(test_reader) == b'S\x00\x00\n\x04L`09O'
assert next(test_reader) == b'S\x00\x00\r\x18\xc2\xe5B\xedS'

In [6]:
grader.check("q1")

ValueError: Tests directory does not exist and no notebook path provided

**Question 2:** Parse system event messages according to the spec.

In [ ]:
def parse_system_event(message):
    return {
        'type': 'S',
        'timestamp': parse_timestamp(read_int(message[3:9])),
        'event': {
            'O': 'Start of Transmissions',
            'S': 'Start of System Hours',
            'Q': 'Start of Market Hours',
            'M': 'End of Market Hours',
            'E': 'End of System Hours',
            'C': 'End of Transmissions',
        }[read_string(message[9:10])]
    }

In [ ]:
assert parse_system_event(b'S\x00\x00\r\x18\xc2\xe5B\xedS') == {
    'type': 'S',
    'timestamp': datetime.datetime(2019, 4, 22, 4, 0, 0, 181),
    'event': 'Start of System Hours',
}

In [ ]:
grader.check("q2")

**Question 3:** Parse quotation messages

In [ ]:
def parse_quotation(message):
    return {
        'type': 'Q',
        'timestamp': parse_timestamp(read_int(message[3:9])),
        'ticker': read_string(message[9:9+8]).strip(),
        'bid_price': read_int(message[18:18+4]) / 10000,
        'bid_size': read_int(message[22:22+4]),
        'ask_price': read_int(message[26:26+4]) / 10000,
        'ask_size': read_int(message[30:30+4]),
    }

In [ ]:
assert parse_quotation(b'Q\x00\x00\r\x18\xc3\xe4\xa9GMSFT    Q\x00\x0c5\x00\x00\x00\x00d\x00\x00\x00\x00\x00\x00\x00\x00') == {
    'type': 'Q',
    'timestamp': datetime.datetime(2019, 4, 22, 4, 0, 0, 16919),
    'ticker': 'MSFT',
    'bid_price': 80.0,
    'bid_size': 100,
    'ask_price': 0.0,
    'ask_size': 0
}

In [ ]:
grader.check("q3")

**Question 4:** Parse a message using its message type (the first character). Use the functions you wrote above.

In [ ]:
def parse_message(message):
    message_type = read_string(message[:1])
    if message_type == 'S':
        return parse_system_event(message)
    elif message_type == 'Q':
        return parse_quotation(message)
    else:
        raise ValueError('Unknown message type: {}'.format(message_type))

In [ ]:
assert parse_message(b'S\x00\x00\n\x04L`09O') == {
    'type': 'S',
    'timestamp': datetime.datetime(2019, 4, 22, 3, 3, 33, 577519),
    'event': 'Start of Transmissions',
}

In [ ]:
assert parse_message(b'Q\x00\x00\r\x18\xc3\xe4\xa9GMSFT    Q\x00\x0c5\x00\x00\x00\x00d\x00\x00\x00\x00\x00\x00\x00\x00') == {
    'type': 'Q',
    'timestamp': datetime.datetime(2019, 4, 22, 4, 0, 0, 16919),
    'ticker': 'MSFT',
    'bid_price': 80.0,
    'bid_size': 100,
    'ask_price': 0.0,
    'ask_size': 0
}

In [ ]:
grader.check("q4")

**Question 5:** Now, write a function to read a QBBO file and yield parsed messages. Use the functions you wrote above.

In [ ]:
def read_qbbo_messages_parsed(path):
    for message in read_qbbo_messages(path):
        yield parse_message(message)

In [ ]:
test_reader = read_qbbo_messages_parsed(QBBO_PATH)
assert next(test_reader) == {
    'type': 'S',
    'timestamp': datetime.datetime(2019, 4, 22, 3, 3, 33, 577519),
    'event': 'Start of Transmissions'
}
assert next(test_reader) == {
    'type': 'S',
    'timestamp': datetime.datetime(2019, 4, 22, 4, 0, 0, 181),
    'event': 'Start of System Hours'
}
assert next(test_reader) == {
    'type': 'Q',
    'timestamp': datetime.datetime(2019, 4, 22, 4, 0, 0, 16919),
    'ticker': 'MSFT',
    'bid_price': 80.0,
    'bid_size': 100,
    'ask_price': 0.0,
    'ask_size': 0
}

In [ ]:
grader.check("q5")

Congratulations! You can now read real marketdata. Although professional trading systems use ITCH (level 2) instead of QBBO (level 1), the protocols are very similar! You can [read ITCH's spec online](https://www.nasdaqtrader.com/content/technicalsupport/specifications/dataproducts/NQTVITCHSpecification.pdf).

Let's do some analysis now that we can read the file.

**Question 6:** Which tickers are included in this file? Write a function that returns a set of ticker names

In [ ]:
def tickers_set(path):
    out = set()
    for messsage in read_qbbo_messages_parsed(path):
        if messsage['type'] == 'Q':
            out.add(messsage['ticker'])
    return out

In [ ]:
assert len(tickers_set(QBBO_PATH)) == 4

In [ ]:
grader.check("q6")

Finally, let's plot the market of `$MSFT`. This part is ungraded:

In [ ]:
X_bid = []
Y_bid = []

X_ask = []
Y_ask = []
for message in read_qbbo_messages_parsed(QBBO_PATH):
    if message['timestamp'].hour < 10 or message['timestamp'].hour > 15:
        continue # Skip most out-of-market-hours messages
    if message['type'] == 'Q' and message['ticker'] == 'MSFT':
        if message['bid_price'] > 0:
            X_bid.append(message['timestamp'])
            Y_bid.append(message['bid_price'])
        if message['ask_price'] > 0:
            X_ask.append(message['timestamp'])
            Y_ask.append(message['ask_price'])
        

import matplotlib.pyplot as plt
plt.figure(figsize=(10, 5))
plt.plot(X_ask, Y_ask, label='Ask', color='blue')
plt.plot(X_bid, Y_bid, label='Bid', color='red')
plt.legend()

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(run_tests=True)